In [347]:
import pandas as pd
import numpy as np
import gensim 
import jieba
from gensim.models import Word2Vec 
import random 
import re
import os 

In [349]:
from datetime import datetime 
import pytz 
  
IST = pytz.timezone('Asia/Kolkata') 
print("start time", datetime.now(IST)) 
print("---------------------------------")

start time 2021-01-02 22:14:39.781555+05:30
---------------------------------


In [350]:
def read_data():
    #dataset1==> ML_data_trial_drug_disease_prot_distinct                
    #data=pd.read_excel("/sas/vidhya/CompoundDb4jV2/ClinicalTrials/MLData/ml/examples//ML_data_trial_drug_disease_prot_distinct.xlsx", 
                     #sheet_name="pass")
    
    #data=pd.read_excel("/sas/vidhya/CompoundDb4jV2/ClinicalTrials/MLData/ml/examples/ML_data_trial_drug_disease_prot_distinct.xlsx", 
                    #sheet_name="fail")
    
    #dataset2==> ML_data_trial_drug_disease_prot_distinct_withctd
    #data=pd.read_excel("/sas/vidhya/CompoundDb4jV2/ClinicalTrials/MLData/ml/examples/ML_data_trial_drug_disease_prot_distinct_withctd.xlsx", 
                     #sheet_name="pass_withctduniprot")
    
    data=pd.read_excel("/sas/vidhya/CompoundDb4jV2/ClinicalTrials/MLData/ml/examples/ML_data_trial_drug_disease_prot_distinct_withctd.xlsx", 
                    sheet_name="fail_withctduniprot")
    
    
    #print(data)
    return data

In [351]:
def SMILES2Vec(data, dims,window_size,negative_size):
#def SMILES2Vec(data, dims,window_size,negative_size):
        SMILES_vec = pd.DataFrame()
        dictionary=[]
        Index = []
        #data= read_data()
        texts = [[word for word in re.findall(r'.{6}',document)] for document in list(data)]
        #print("texts",texts)
        model = Word2Vec(texts,size=dims,window=window_size,min_count=1,negative=negative_size,sg=1,sample=0.001,hs=1,workers=4)
        vectors = pd.DataFrame([model[word] for word in (model.wv.vocab)])
        print("vectors",vectors)
        vectors['Word'] = list(model.wv.vocab)
        #print("vectors--",vectors)

        for i in range(len(data)):
            Index.append(i)
        # 'Word' segmentation
        for i in range(len(texts)):
            i_word=[]         
            for w in range(len(texts[i])):
                i_word.append(Index[i])    
            dictionary.extend(i_word)
        SMILES_vec['Id'] = dictionary
        
        #print('SMILES_vec id',SMILES_vec['Id'])
        
        # substructure vectors generation
        dictionary=[]
        for i in range(len(texts)):
            i_word=[]         
            for w in range(len(texts[i])):
                i_word.append(texts[i][w])    
            dictionary.extend(i_word)
        SMILES_vec['Word'] = dictionary
        
        #print('SMILES_vec Word',SMILES_vec['Word'])
        
        del dictionary,i_word
        
        SMILES_vec = SMILES_vec.merge(vectors,on='Word', how='left')
        SMILES_vec.columns = ['Id']+['word']+["vec_{0}".format(i) for i in range(0,dims)]
        print('SMILES_vec--',SMILES_vec['word'])

        SMILES_vec=SMILES_vec.drop('word',axis=1)
   
        #Molecular Structure  Representation

        name = ["vec_{0}".format(i) for i in range(0,dims)]
        drug_embeddings = pd.DataFrame(SMILES_vec.groupby(['Id'])[name].agg('mean')).reset_index()
        drug_embeddings.columns=["Index"]+["mean_ci_{0}".format(i) for i in range(0,dims)]
        
        #drug_embeddings.to_csv('/sas/vidhya/CompoundDb4jV2/ClinicalTrials/MLData/ml/SPVecout/dataset1/ML_data_trial_drug_embeddings_pass_10.csv',index=False)
        #drug_embeddings.to_csv('/sas/vidhya/CompoundDb4jV2/ClinicalTrials/MLData/ml/SPVecout/dataset1/ML_data_trial_drug_embeddings_fail_10.csv',index=False)
        
        #drug_embeddings.to_csv('/sas/vidhya/CompoundDb4jV2/ClinicalTrials/MLData/ml/SPVecout/dataset2/ML_dataset2_trial_drug_embeddings_pass_5.csv',index=False)
        drug_embeddings.to_csv('/sas/vidhya/CompoundDb4jV2/ClinicalTrials/MLData/ml/SPVecout/dataset2/ML_dataset2_trial_drug_embeddings_fail_5.csv',index=False)
        

        #print(drug_embeddings)
        return SMILES_vec,drug_embeddings

In [352]:
def ProtVec(data,dims,window_size,negative_size):
        Prot_vec = pd.DataFrame()
        dictionary=[]
        Index = []
        #data=self.read_data()
        texts = [[word for word in re.findall(r'.{3}',document)] for document in list(data)]
        model = Word2Vec(texts,size=dims,window=window_size,min_count=1,negative=negative_size,sg=1,sample=0.001,hs=1,workers=4)
        vectors = pd.DataFrame([model[word] for word in (model.wv.vocab)])
        vectors['Word'] = list(model.wv.vocab)

        for i in range(len(data)):
            Index.append(i)
        # 'Word'segmentation
        for i in range(len(texts)):
            i_word=[]         
            for w in range(len(texts[i])):
                i_word.append(Index[i])    
            dictionary.extend(i_word)
        Prot_vec['Id'] = dictionary
        
        # word vectors generation
        dictionary=[]
        for i in range(len(texts)):
            i_word=[]         
            for w in range(len(texts[i])):
                i_word.append(texts[i][w])    
            dictionary.extend(i_word)
        Prot_vec['Word'] = dictionary
        
        del dictionary,i_word
        
        Prot_vec = Prot_vec.merge(vectors,on='Word', how='left')
        Prot_vec.columns = ['Id']+['word']+["vec_{0}".format(i) for i in range(0,dims)]
        #Prot_vec= Prot_vec.drop('Word',axis=1)
        Prot_vec= Prot_vec.drop('word',axis=1)

        #Protein Sequence Representation

        name = ["vec_{0}".format(i) for i in range(0,dims)]
        Prot_embeddings = pd.DataFrame(Prot_vec.groupby(['Id'])[name].agg('mean')).reset_index()
        Prot_embeddings.columns=["Index"]+["mean_ci_{0}".format(i) for i in range(0,dims)]
        
        #Prot_embeddings.to_csv('/sas/vidhya/CompoundDb4jV2/ClinicalTrials/MLData/ml/SPVecout/dataset1/ML_data_trial_Prot_embeddings_pass_10.csv',index=False)
        #Prot_embeddings.to_csv('/sas/vidhya/CompoundDb4jV2/ClinicalTrials/MLData/ml/SPVecout/dataset1/ML_data_trial_Prot_embeddings_fail_10.csv',index=False)

        #Prot_embeddings.to_csv('/sas/vidhya/CompoundDb4jV2/ClinicalTrials/MLData/ml/SPVecout/dataset2/ML_dataset2_trial_Prot_embeddings_pass_5.csv',index=False)
        Prot_embeddings.to_csv('/sas/vidhya/CompoundDb4jV2/ClinicalTrials/MLData/ml/SPVecout/dataset2/ML_dataset2_trial_Prot_embeddings_fail_5.csv',index=False)

        return Prot_vec,Prot_embeddings


In [353]:
data=read_data()
#data.head()

In [354]:
listTrialIds=[x for x in data[data.columns[1]]]
print(len(listTrialIds))

1629


In [355]:
data.head()

,index,id,trials url,identifiers,recruitment_status,has_published_results,status,registration_date,source_id,intervention_id,...,gene_sequence,aminoacid_sequence,condition id,condition name,condition_aasequence,condition_amino,condition,study_type,study_phase,Label
0,0,9ea0c8bf-782c-4098-9d9c-00a03707d0ac,https://explorer.opentrials.net/trials/9ea0c8b...,"{""nct"": ""NCT01061372""}",not_recruiting,f,withdrawn,2010-02-01 00:00:00,nct,eaa19797-d103-4162-9233-19df89b6b79a,...,|>lcl||BSEQ0016179||Voltage-dependent_calcium_...,|>lcl||BSEQ0037060||Voltage-dependent_calcium_...,ec1e046e-8c18-11e6-a776-0242ac12000b,restless legs syndrome,SEQUENCE 495 AA; 54254 MW; 8E611BDC3BC824C...,MSMLVVFLLLWGVTWGPVTEAAIFYETQPSLWAESESLLKPLANVT...,restless legs syndromeMSMLVVFLLLWGVTWGPVTEAAIF...,Interventional,"{""Phase 3""}",Fail
1,4,3ccd507f-08e5-493e-8687-6b7bf86cbc58,https://explorer.opentrials.net/trials/3ccd507...,"{""nct"": ""NCT00845910""}",not_recruiting,f,terminated,2009-02-17 00:00:00,nct,ca02cb56-e5ec-4421-b46c-4ef2da6e4a29,...,|>lcl||BSEQ0010672||Nuclear_receptor_subfamily...,|>lcl||BSEQ0001903||Nuclear_receptor_subfamily...,d5f54bb2-8c17-11e6-a776-0242ac12000b,breast cancer,SEQUENCE 495 AA; 54254 MW; 8E611BDC3BC824C...,MSMLVVFLLLWGVTWGPVTEAAIFYETQPSLWAESESLLKPLANVT...,breast cancerMSMLVVFLLLWGVTWGPVTEAAIFYETQPSLWA...,Interventional,"{""Phase 2""}",Fail
2,8,51b5dd3a-3031-4f34-9f33-ac7944429239,https://explorer.opentrials.net/trials/51b5dd3...,"{""isrctn"": ""ISRCTN63558875""}",not_recruiting,f,terminated,2006-12-26 00:00:00,isrctn,dbdea678-d80b-46e4-86f8-37cf1d7f20e0,...,|>lcl||BSEQ0017203||Aryl_hydrocarbon_receptor_...,|>lcl||BSEQ0017202||Aryl_hydrocarbon_receptor ...,ec70d49a-8c23-11e6-be70-0242ac12000f,pulmonary hypertension,SEQUENCE 495 AA; 54254 MW; 8E611BDC3BC824C...,MSMLVVFLLLWGVTWGPVTEAAIFYETQPSLWAESESLLKPLANVT...,pulmonary hypertensionMSMLVVFLLLWGVTWGPVTEAAIF...,Interventional,"{""Phase 3""}",Fail
3,9,aacf04ec-6cfd-4f94-9eab-38ed5be2915d,https://explorer.opentrials.net/trials/aacf04e...,"{""nct"": ""NCT00337740""}",not_recruiting,f,terminated,2006-06-14 00:00:00,nct,b22bf162-30b1-456e-8b62-4de998626ccd,...,|>lcl||BSEQ0010189||Beta-1_adrenergic_receptor...,|>lcl||BSEQ0010188||Beta-1_adrenergic_receptor...,087b4ad9-02a2-4fc6-8855-b3c7263381cc,esophageal varices,SEQUENCE 672 AA; 75144 MW; 36006739EE4857D...,MSKEERPGREEILECQVMWEPDSKKNTQMDRFRAAVGAACGLALES...,esophageal varicesMSKEERPGREEILECQVMWEPDSKKNTQ...,Interventional,"{""Phase 3""}",Fail
4,11,11db3682-e77c-4cd0-ab35-e488553e93ef,https://explorer.opentrials.net/trials/11db368...,"{""nct"": ""NCT00895960""}",not_recruiting,f,terminated,2009-05-07 00:00:00,nct,954cc411-a1dc-432f-8aa3-0b73ba96b893,...,|>lcl||BSEQ0019022||Ephrin_type-A_receptor_2_(...,|>lcl||BSEQ0037092||Ephrin_type-A_receptor_2 M...,dfffe57a-cf60-4c95-b72d-07086357e323,brain diseases,SEQUENCE 495 AA; 54254 MW; 8E611BDC3BC824C...,MSMLVVFLLLWGVTWGPVTEAAIFYETQPSLWAESESLLKPLANVT...,brain diseasesMSMLVVFLLLWGVTWGPVTEAAIFYETQPSLW...,Interventional,"{""Phase 1""}",Fail


In [356]:
data_smi=data['SMILES']
print("data_smi",data_smi)
#drug_vec, drug_em=SMILES2Vec(data_smi,5,5,5)

drug_vec, drug_em=SMILES2Vec(data_smi,5,5,5)
print("drug_vec",drug_vec)
print("drug_em",drug_em)


data_smi 0                                  CC(C)C[C@H](CN)CC(O)=O
1       [H][C@@]1(C[C@@]2(O)[C@@H](OC(=O)C3=CC=CC=C3)[...
2       CC(C)C1=C(C(=O)NC2=CC=CC=C2)C(=C(N1CC[C@@H](O)...
3                        CC(C)NCC(O)COC1=CC=CC2=C1C=CC=C2
4       CC1=NC(NC2=NC=C(S2)C(=O)NC2=C(C)C=CC=C2Cl)=CC(...
                              ...                        
1624        CCC1=CN=C(CCOC2=CC=C(CC3SC(=O)NC3=O)C=C2)C=C1
1625               CCN(CC)CCCC(C)NC1=CC=NC2=CC(Cl)=CC=C12
1626    CN1[C@H]2CC[C@@H]1C[C@@H](C2)OC(=O)C(CO)C1=CC=...
1627    [H][C@]12[C@H](OC(=O)C3=CC=CC=C3)[C@]3(O)C[C@H...
1628               O=C1N(C2CCC(=O)NC2=O)C(=O)C2=CC=CC=C12
Name: SMILES, Length: 1629, dtype: object
vectors              0         1         2         3         4
0     0.319796 -0.320078 -0.517204  0.089317  1.303091
1    -0.216005 -0.348614 -1.706201 -0.570533  1.381539
2    -0.004345 -0.473094 -0.821821 -0.359202  0.660041
3     0.803858  0.152955 -1.153243 -0.701445  0.791884
4     0.313192 -0.191293 -2.

<ipython-input-351-6e54062e301e>:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  vectors = pd.DataFrame([model[word] for word in (model.wv.vocab)])


In [357]:
prot_seq=data['amino']
print("prot_seq",prot_seq)
prot_vec, prot_em=ProtVec(prot_seq,5,5, 5)
print("prot_vec",prot_vec)
print("prot_em",prot_em)

prot_seq 0       MAAGCLLALTLTLFQSLLIGPSSEEPFPSAVTIKSWVDKMQEDLVT...
1       MEVRPKESWNHADFVHCEDTESVPGKPSVNADEEVGGPQICRVCGD...
2       MNSSSANITYASRKRRKPVQKTVKPIPAEGIKSNPSKRHRDRLNTE...
3       MGAGVLVLGASEPGNLSSAAPLPDGAATAARLLVPASPPASLLPPA...
4       MELQAARACFALLWGCALAAAAAAQGKEVVLLDFAAAGGELGWLTH...
                              ...                        
1624    MGETLGDSPIDPESDSFTDTLSANISQEMTMVDTEMPFWPTNFGIS...
1625    MSSSSWLLLSLVAVTAAQSTIEEQAKTFLDKFNHEAEDLFYQSSLA...
1626    MNTSAPPAVSPNITVLAPGKGPWQVAFIGITTGLLSLATVTGNLLV...
1627    MEVRPKESWNHADFVHCEDTESVPGKPSVNADEEVGGPQICRVCGD...
1628    MAEDDPYLGRPEQMFHLDPSLTHTIFNPEVFQPQMALPTDGPYLQI...
Name: amino, Length: 1629, dtype: object


<ipython-input-352-7dabc5790476>:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  vectors = pd.DataFrame([model[word] for word in (model.wv.vocab)])


prot_vec            Id     vec_0     vec_1     vec_2     vec_3     vec_4
0           0  0.492077 -0.707363 -0.657483 -0.045242  0.377446
1           0  0.387585 -0.513367 -0.930683  0.000071 -0.054778
2           0  0.612182 -0.744922 -0.639191 -0.493805 -0.059993
3           0  0.107714 -0.563485 -0.945417  0.016647  0.003451
4           0  0.219442 -0.454149 -0.553099 -0.072370  0.881148
...       ...       ...       ...       ...       ...       ...
1331996  1628 -0.311943 -0.739892 -0.713801 -0.418173  0.041561
1331997  1628 -0.182598 -0.999748 -0.421649 -1.124088 -0.016748
1331998  1628 -0.093521 -0.755225 -0.755738 -0.350849 -0.032329
1331999  1628  0.790832 -1.002998 -0.316661 -0.456849  0.719312
1332000  1628  0.988836 -1.153060 -1.334063  0.374269 -0.292052

[1332001 rows x 6 columns]
prot_em       Index  mean_ci_0  mean_ci_1  mean_ci_2  mean_ci_3  mean_ci_4
0         0   0.054463  -0.842223  -0.683866  -0.324841   0.281277
1         1  -0.153908  -0.824745  -0.808528  -0.6206

In [361]:
def Dis2Vec(data,dims,window_size,negative_size):
        Prot_vec = pd.DataFrame()
        dictionary=[]
        Index = []
        #data=self.read_data()
        texts = [[word for word in re.findall(r'.{3}',str(document))] for document in list(data)]
        model = Word2Vec(texts,size=dims,window=window_size,min_count=1,negative=negative_size,sg=1,sample=0.001,hs=1,workers=4)
        vectors = pd.DataFrame([model[word] for word in (model.wv.vocab)])
        vectors['Word'] = list(model.wv.vocab)

        for i in range(len(data)):
            Index.append(i)
        # 'Word'segmentation
        for i in range(len(texts)):
            i_word=[]         
            for w in range(len(texts[i])):
                i_word.append(Index[i])    
            dictionary.extend(i_word)
        Prot_vec['Id'] = dictionary
        
        # word vectors generation
        dictionary=[]
        for i in range(len(texts)):
            i_word=[]         
            for w in range(len(texts[i])):
                i_word.append(texts[i][w])    
            dictionary.extend(i_word)
        Prot_vec['Word'] = dictionary
        
        del dictionary,i_word
        
        Prot_vec = Prot_vec.merge(vectors,on='Word', how='left')
        Prot_vec.columns = ['Id']+['word']+["vec_{0}".format(i) for i in range(0,dims)]
        #Prot_vec= Prot_vec.drop('Word',axis=1)
        Prot_vec= Prot_vec.drop('word',axis=1)

        #Protein Sequence Representation

        name = ["vec_{0}".format(i) for i in range(0,dims)]
        Prot_embeddings = pd.DataFrame(Prot_vec.groupby(['Id'])[name].agg('mean')).reset_index()
        Prot_embeddings.columns=["Index"]+["mean_ci_{0}".format(i) for i in range(0,dims)]
        
        #Prot_embeddings.to_csv('/sas/vidhya/CompoundDb4jV2/ClinicalTrials/MLData/ml/SPVecout/dataset1/ML_data_trial_Dis_embeddings_pass_10.csv',index=False)
        #Prot_embeddings.to_csv('/sas/vidhya/CompoundDb4jV2/ClinicalTrials/MLData/ml/SPVecout/dataset1/ML_data_trial_Dis_embeddings_fail_10.csv',index=False)
        
        #Prot_embeddings.to_csv('/sas/vidhya/CompoundDb4jV2/ClinicalTrials/MLData/ml/SPVecout/dataset2/ML_dataset2_trial_Dis_embeddings_pass_5.csv',index=False)
        Prot_embeddings.to_csv('/sas/vidhya/CompoundDb4jV2/ClinicalTrials/MLData/ml/SPVecout/dataset2/ML_dataset2_trial_Dis_embeddings_fail_5.csv',index=False)

        
        #print("Prot_embeddings", Prot_embeddings)
        
        #print("type Prot_embeddings", type(Prot_embeddings))

        return Prot_vec,Prot_embeddings


In [362]:
def Dis2VecAmino(data,dims,window_size,negative_size):
        Prot_vec = pd.DataFrame()
        dictionary=[]
        Index = []
        #data=self.read_data()
        texts = [[word for word in re.findall(r'.{3}',str(document))] for document in list(data)]
        model = Word2Vec(texts,size=dims,window=window_size,min_count=1,negative=negative_size,sg=1,sample=0.001,hs=1,workers=4)
        vectors = pd.DataFrame([model[word] for word in (model.wv.vocab)])
        vectors['Word'] = list(model.wv.vocab)

        for i in range(len(data)):
            Index.append(i)
        # 'Word'segmentation
        for i in range(len(texts)):
            i_word=[]         
            for w in range(len(texts[i])):
                i_word.append(Index[i])    
            dictionary.extend(i_word)
        Prot_vec['Id'] = dictionary
        
        # word vectors generation
        dictionary=[]
        for i in range(len(texts)):
            i_word=[]         
            for w in range(len(texts[i])):
                i_word.append(texts[i][w])    
            dictionary.extend(i_word)
        Prot_vec['Word'] = dictionary
        
        del dictionary,i_word
        
        Prot_vec = Prot_vec.merge(vectors,on='Word', how='left')
        Prot_vec.columns = ['Id']+['word']+["vec_{0}".format(i) for i in range(0,dims)]
        #Prot_vec= Prot_vec.drop('Word',axis=1)
        Prot_vec= Prot_vec.drop('word',axis=1)

        #Protein Sequence Representation

        name = ["vec_{0}".format(i) for i in range(0,dims)]
        Prot_embeddings = pd.DataFrame(Prot_vec.groupby(['Id'])[name].agg('mean')).reset_index()
        Prot_embeddings.columns=["Index"]+["mean_ci_{0}".format(i) for i in range(0,dims)]
        
        #Prot_embeddings.to_csv('/sas/vidhya/CompoundDb4jV2/ClinicalTrials/MLData/ml/SPVecout/dataset1/ML_data_trial_Dis_embeddings_pass_10.csv',index=False)
        #Prot_embeddings.to_csv('/sas/vidhya/CompoundDb4jV2/ClinicalTrials/MLData/ml/SPVecout/dataset1/ML_data_trial_Dis_embeddings_fail_10.csv',index=False)
        
        #Prot_embeddings.to_csv('/sas/vidhya/CompoundDb4jV2/ClinicalTrials/MLData/ml/SPVecout/dataset2/ML_dataset2_trial_Disamino_embeddings_pass_5.csv',index=False)
        Prot_embeddings.to_csv('/sas/vidhya/CompoundDb4jV2/ClinicalTrials/MLData/ml/SPVecout/dataset2/ML_dataset2_trial_Disamino_embeddings_fail_5.csv',index=False)

        
        #print("Prot_embeddings", Prot_embeddings)
        
        #print("type Prot_embeddings", type(Prot_embeddings))

        return Prot_vec,Prot_embeddings


In [363]:
#data_dis=data['condition'] #dataset2
data_dis=data['condition name']

print("data_dis",data_dis)
#print("list(data)", list(data_dis))
dis_vec, dis_em =Dis2Vec(data_dis,5,5, 5)
print("dis_vec",dis_vec)

print("dis_em",dis_em)


data_dis 0              restless legs syndrome
1                       breast cancer
2              pulmonary hypertension
3                  esophageal varices
4                      brain diseases
                    ...              
1624    amyotrophic lateral sclerosis
1625              malaria, falciparum
1626                        amblyopia
1627                    breast cancer
1628                    endometriosis
Name: condition name, Length: 1629, dtype: object
dis_vec         Id     vec_0     vec_1     vec_2     vec_3     vec_4
0        0 -0.377524 -1.006231 -0.274066  0.983294 -0.775609
1        0  0.162129 -0.248938 -0.139935  0.338873 -0.280649
2        0 -0.283600 -0.422849 -0.663455  0.800679 -0.561355
3        0 -0.088398 -0.173331 -0.093878  0.343989 -0.277007
4        0 -0.030756 -0.318003 -0.212371  0.407957 -0.655173
...    ...       ...       ...       ...       ...       ...
9027  1627  0.071678 -0.080001 -0.467253  0.417189 -0.147034
9028  1628 -0.662600 -0.502

<ipython-input-361-7cb953007598>:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  vectors = pd.DataFrame([model[word] for word in (model.wv.vocab)])


In [364]:
#add one more for dis sequence 
data_dis_amino=data['condition_amino'] #dataset2

print("data_dis_amino",data_dis_amino)
#print("list(data)", list(data_dis))
dis_vecamino, dis_embamino =Dis2VecAmino(data_dis_amino,5,5,5)
print("dis_vecamino",dis_vecamino)

print("dis_embamino",dis_embamino)


data_dis_amino 0       MSMLVVFLLLWGVTWGPVTEAAIFYETQPSLWAESESLLKPLANVT...
1       MSMLVVFLLLWGVTWGPVTEAAIFYETQPSLWAESESLLKPLANVT...
2       MSMLVVFLLLWGVTWGPVTEAAIFYETQPSLWAESESLLKPLANVT...
3       MSKEERPGREEILECQVMWEPDSKKNTQMDRFRAAVGAACGLALES...
4       MSMLVVFLLLWGVTWGPVTEAAIFYETQPSLWAESESLLKPLANVT...
                              ...                        
1624    MSMLVVFLLLWGVTWGPVTEAAIFYETQPSLWAESESLLKPLANVT...
1625    MSMLVVFLLLWGVTWGPVTEAAIFYETQPSLWAESESLLKPLANVT...
1626    MSMLVVFLLLWGVTWGPVTEAAIFYETQPSLWAESESLLKPLANVT...
1627    MSMLVVFLLLWGVTWGPVTEAAIFYETQPSLWAESESLLKPLANVT...
1628    MSMLVVFLLLWGVTWGPVTEAAIFYETQPSLWAESESLLKPLANVT...
Name: condition_amino, Length: 1629, dtype: object


<ipython-input-362-839db48a0381>:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  vectors = pd.DataFrame([model[word] for word in (model.wv.vocab)])


dis_vecamino           Id     vec_0     vec_1     vec_2     vec_3     vec_4
0          0  2.405303  0.109488 -0.954639  0.611128  1.976193
1          0  1.797076 -0.004067 -0.969261  0.581380  1.782011
2          0  1.125430  0.988612 -0.532507  0.499366  1.700402
3          0  2.153718 -0.139615 -1.250534  0.778814  1.946274
4          0  2.539138 -0.318532 -1.361637  0.744989  2.181988
...      ...       ...       ...       ...       ...       ...
337805  1628  1.081483  0.057964 -0.978934  0.834567  0.527333
337806  1628  1.310171  0.035624 -0.470130  0.319061  0.195352
337807  1628  2.189234  1.211433 -1.084351  0.616599 -0.896406
337808  1628  2.123335  0.279751 -2.380239  1.793411  0.162198
337809  1628  2.884855  1.317716 -1.687300  1.041397 -0.786615

[337810 rows x 6 columns]
dis_embamino       Index  mean_ci_0  mean_ci_1  mean_ci_2  mean_ci_3  mean_ci_4
0         0   2.785180   1.302161  -1.533917   0.604852   1.553688
1         1   2.785180   1.302161  -1.533917   0.604852  

In [365]:
print("done---")

done---


In [366]:
#dataset1
#df1 = pd.read_csv("/sas/vidhya/CompoundDb4jV2/ClinicalTrials/MLData/ml/SPVecout/dataset1/ML_data_trial_drug_embeddings_pass_10.csv") 
#df2 = pd.read_csv("/sas/vidhya/CompoundDb4jV2/ClinicalTrials/MLData/ml/SPVecout/dataset1/ML_data_trial_Prot_embeddings_pass_10.csv") 
#df3 = pd.read_csv("/sas/vidhya/CompoundDb4jV2/ClinicalTrials/MLData/ml/SPVecout/dataset1/ML_data_trial_Dis_embeddings_pass_10.csv")


#df1 = pd.read_csv("/sas/vidhya/CompoundDb4jV2/ClinicalTrials/MLData/ml/SPVecout/dataset1/ML_data_trial_drug_embeddings_fail_10.csv") 
#df2 = pd.read_csv("/sas/vidhya/CompoundDb4jV2/ClinicalTrials/MLData/ml/SPVecout/dataset1/ML_data_trial_Prot_embeddings_fail_10.csv") 
#df3 = pd.read_csv("/sas/vidhya/CompoundDb4jV2/ClinicalTrials/MLData/ml/SPVecout/dataset1/ML_data_trial_Dis_embeddings_fail_10.csv")


#dataset2
'''
df1 = pd.read_csv("/sas/vidhya/CompoundDb4jV2/ClinicalTrials/MLData/ml/SPVecout/dataset2/ML_dataset2_trial_drug_embeddings_pass_5.csv") 
df2 = pd.read_csv("/sas/vidhya/CompoundDb4jV2/ClinicalTrials/MLData/ml/SPVecout/dataset2/ML_dataset2_trial_Prot_embeddings_pass_5.csv") 
df3 = pd.read_csv("/sas/vidhya/CompoundDb4jV2/ClinicalTrials/MLData/ml/SPVecout/dataset2/ML_dataset2_trial_Dis_embeddings_pass_5.csv")
df4 = pd.read_csv("/sas/vidhya/CompoundDb4jV2/ClinicalTrials/MLData/ml/SPVecout/dataset2/ML_dataset2_trial_Disamino_embeddings_pass_5.csv")

'''
df1 = pd.read_csv("/sas/vidhya/CompoundDb4jV2/ClinicalTrials/MLData/ml/SPVecout/dataset2/ML_dataset2_trial_drug_embeddings_fail_5.csv") 
df2 = pd.read_csv("/sas/vidhya/CompoundDb4jV2/ClinicalTrials/MLData/ml/SPVecout/dataset2/ML_dataset2_trial_Prot_embeddings_fail_5.csv") 
df3 = pd.read_csv("/sas/vidhya/CompoundDb4jV2/ClinicalTrials/MLData/ml/SPVecout/dataset2/ML_dataset2_trial_Dis_embeddings_fail_5.csv")
df4 = pd.read_csv("/sas/vidhya/CompoundDb4jV2/ClinicalTrials/MLData/ml/SPVecout/dataset2/ML_dataset2_trial_Disamino_embeddings_fail_5.csv")


In [372]:
#df4 = [df1, df2, df3]
#result = pd.concat(df4, axis=1)

#df5 = [df1, df2, df3, df4] #dataset2

df5 = [df1, df2, df4] #dataset2
result = pd.concat(df5, axis=1)

#print(result)

In [373]:
print(result)

      Index  mean_ci_0  mean_ci_1  mean_ci_2  mean_ci_3  mean_ci_4  Index  \
0       0.0   0.033149  -0.380595  -1.015075  -0.280139   1.114891      0   
1       1.0  -0.598140  -0.614370  -2.837381  -0.479334   1.661100      1   
2       2.0   0.518176  -0.315953  -0.826430  -0.046258   1.964468      2   
3       3.0   0.555290   0.263048  -0.939904   0.601318   0.298414      3   
4       4.0  -1.675760  -1.465615  -2.033607   1.465440   1.452184      4   
...     ...        ...        ...        ...        ...        ...    ...   
1624    NaN        NaN        NaN        NaN        NaN        NaN   1624   
1625    NaN        NaN        NaN        NaN        NaN        NaN   1625   
1626    NaN        NaN        NaN        NaN        NaN        NaN   1626   
1627    NaN        NaN        NaN        NaN        NaN        NaN   1627   
1628    NaN        NaN        NaN        NaN        NaN        NaN   1628   

      mean_ci_0  mean_ci_1  mean_ci_2  mean_ci_3  mean_ci_4  Index  mean_ci

In [374]:
#result = result.drop(result.columns[[0, 4, 8]], axis=1)  #n=3

#result = result.drop(result.columns[[0, 6, 12]], axis=1)  #n=5

#result = result.drop(result.columns[[0, 51, 102]], axis=1) #n=50

#result = result.drop(result.columns[[0, 101, 202]], axis=1) #n=100


#result = result.drop(result.columns[[0, 11, 22]], axis=1) #n=10


result = result.drop(result.columns[[0, 6, 12]], axis=1)  #n=5


print(result)

      mean_ci_0  mean_ci_1  mean_ci_2  mean_ci_3  mean_ci_4  mean_ci_0  \
0      0.033149  -0.380595  -1.015075  -0.280139   1.114891   0.054463   
1     -0.598140  -0.614370  -2.837381  -0.479334   1.661100  -0.153908   
2      0.518176  -0.315953  -0.826430  -0.046258   1.964468   0.053401   
3      0.555290   0.263048  -0.939904   0.601318   0.298414   0.184471   
4     -1.675760  -1.465615  -2.033607   1.465440   1.452184   0.006953   
...         ...        ...        ...        ...        ...        ...   
1624        NaN        NaN        NaN        NaN        NaN   0.138373   
1625        NaN        NaN        NaN        NaN        NaN   0.104726   
1626        NaN        NaN        NaN        NaN        NaN   0.161943   
1627        NaN        NaN        NaN        NaN        NaN  -0.154731   
1628        NaN        NaN        NaN        NaN        NaN  -0.013935   

      mean_ci_1  mean_ci_2  mean_ci_3  mean_ci_4  mean_ci_0  mean_ci_1  \
0     -0.842223  -0.683866  -0.324841

In [370]:
ids=[pd.Series(listTrialIds),result]
result= pd.concat(ids, axis=1)

#result.insert(1, 'label', '0')
result.insert(1, 'label', '1')


#result.to_csv('/sas/vidhya/CompoundDb4jV2/ClinicalTrials/MLData/ml/SPVecout/dataset1/ML_data_trial_all_embeddings_pass_5.csv',index=False)
#result.to_csv('/sas/vidhya/CompoundDb4jV2/ClinicalTrials/MLData/ml/SPVecout/dataset1/ML_data_trial_all_embeddings_fail_5.csv',index=False)


#dataset2
#result.to_csv('/sas/vidhya/CompoundDb4jV2/ClinicalTrials/MLData/ml/SPVecout/dataset2/ML_dataset2_trial_all_embeddings_pass_5.csv',index=False)
result.to_csv('/sas/vidhya/CompoundDb4jV2/ClinicalTrials/MLData/ml/SPVecout/dataset2/ML_dataset2_trial_all_embeddings_fail_5.csv',index=False)


In [371]:
print("end time", datetime.now(IST)) 
print("---------------------------------")

end time 2021-01-02 22:17:45.055142+05:30
---------------------------------
